In [ ]:
#installing faiss for semantic search https://github.com/facebookresearch/faiss
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

--2023-02-23 07:09:12--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=600&X-Amz-Date=20230223T070912Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEEsaCXVzLWVhc3QtMSJHMEUCIHx4IdVpBwhQLMpXmVZ8YMcZ7ing%2B767XJJlefVDmDgSAiEAvQza3l5Rj%2FAPV9TQPAqER13eCgSXJSICrWqLIfmBhbcq1QQI5P%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw0NTU4NjQwOTgzNz

In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.8 MB/s eta 0:00:00


##Import packages

In [ ]:
#Ignore Warning
import warnings
warnings.filterwarnings("ignore") 
# set up notebook to display multiple outputs in one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from tqdm.notebook import tqdm
import argparse
from glob import glob
import faiss
from multiprocessing import Pool, cpu_count
from math import ceil
from collections import defaultdict
from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate,Conv1D,MaxPool1D,Dropout

## Mount Google Drive to Colab environment

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/
!pwd
#!ls
print('Working Directory')
print(os.getcwd())
work_dir = "/content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/"

In [ ]:
#loading question and answer embeddings of train and validation data from disk
train_embeds=pd.read_csv(work_dir + 'data/embedding/re_train_embeds.csv')
val_embeds=pd.read_csv(work_dir + 'data/embedding/re_val_embeds.csv')
test_embeds=pd.read_csv(work_dir + 'data/embedding/re_test_embeds.csv')

In [ ]:
train_embeds.head()

,short_question,short_answer,short_question_pre,short_answer_pre,tags,label,short_answer_embed_numpy,short_question_embed_numpy,question_embeds,answer_embeds
0,sex felt hard painful snap inside penis halfwa...,penile fracture significant chance develop sca...,while having sex i felt a hard painful snap in...,you have had a penile fracture and there is a ...,"['sexual intercourse', 'penis', 'pain', 'lump']",1,[[ 3.36371452e-01 9.03653979e-01 -9.45858717e...,[[-0.6047527 -0.94729406 -0.91843134 -0.78592...,"[-0.6047527, -0.94729406, -0.91843134, -0.7859...","[3.36371452e-01, 9.03653979e-01, -9.45858717e-..."
1,spraying furniture rubbing alcohol cleaning ki...,necessary human head louse egg live long furni...,will spraying furniture with rubbing alcohol a...,this is not necessary human head lice and thei...,['lice'],1,[[ 6.9667971e-01 1.2845362e+00 -1.5401514e-01...,[[-0.1739369 -0.8185432 0.38514182 -0.95360...,"[-0.1739369, -0.8185432, 0.38514182, -0.953606...","[6.9667971e-01, 1.2845362e+00, -1.5401514e-01,..."
2,neuroleptic drug parkinson disease,neuroleptic drug also called major tranquilize...,what are neuroleptic drugs for parkinson is di...,neuroleptic drugs also called major tranquiliz...,"[""parkinson's disease"", 'drug']",1,[[ 0.29717866 1.4708629 -0.56074375 -0.64780...,[[-2.93873221e-01 -8.65725994e-01 -3.97973090e...,"[-2.93873221e-01, -8.65725994e-01, -3.97973090...","[0.29717866, 1.4708629, -0.56074375, -0.647809..."
3,peanut help prevent headache told eat handful ...,apple day keep doctor away heard handful peanu...,do peanuts help prevent headaches have been to...,an apple a day keeps the doctor away but i hav...,"['peanut', 'headache']",1,[[ 0.22861195 0.99265194 -0.75949425 -0.22097...,[[-0.23108998 -0.8298196 -0.74065125 -0.43363...,"[-0.23108998, -0.8298196, -0.74065125, -0.4336...","[0.22861195, 0.99265194, -0.75949425, -0.22097..."
4,osteoarthritis diagnosed,doctor make diagnosis oa based physical exam h...,how is osteoarthritis diagnosed,doctors make a diagnosis of oa based on a phys...,['osteoarthritis'],1,[[-0.10100315 0.81206137 -0.6272399 0.94998...,[[-0.6990545 -0.92379653 -0.74066365 0.95988...,"[-0.6990545, -0.92379653, -0.74066365, 0.95988...","[-0.10100315, 0.81206137, -0.6272399, 0.949985..."


In [ ]:
#function to preprocess question and answer embeddings post loading from disk
def preprocess_embeds(embed):
  output=[]
  embed=embed[2:-2]
  embed=embed.replace('\n','')
  for i in embed.split(' '):
    if i!=' ' and i!='':
      output.append(i)
  return (output)

In [ ]:
#preprocessing embeddings post loading from disk for train data
tqdm.pandas()
train_embeds['question_embeds']=train_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
train_embeds['answer_embeds']=train_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

  0%|          | 0/21398 [00:00<?, ?it/s]

  0%|          | 0/21398 [00:00<?, ?it/s]

In [ ]:
#preprocessing embeddings post loading from disk for validation data
tqdm.pandas()
val_embeds['question_embeds']=val_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
val_embeds['answer_embeds']=val_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

  0%|          | 0/2377 [00:00<?, ?it/s]

  0%|          | 0/2377 [00:00<?, ?it/s]

In [ ]:
#preprocessing embeddings post loading from disk for test data
tqdm.pandas()
test_embeds['question_embeds']=test_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
test_embeds['answer_embeds']=test_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

  0%|          | 0/2642 [00:00<?, ?it/s]

  0%|          | 0/2642 [00:00<?, ?it/s]

In [ ]:
#function to l2 normalize each embedding
def normalize(x):
  x=np.array(x,dtype=np.float64)
  return (x)/(np.linalg.norm(x))

In [ ]:
#normalizing train question and answer embeddings
train_embeds['question_embeds_norm']=train_embeds['question_embeds'].apply(lambda x: normalize(x))
train_embeds['answer_embeds_norm']=train_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [ ]:
train_embeds.head()

,short_question,short_answer,short_question_pre,short_answer_pre,tags,label,short_answer_embed_numpy,short_question_embed_numpy,question_embeds,answer_embeds,question_embeds_norm,answer_embeds_norm
0,sex felt hard painful snap inside penis halfwa...,penile fracture significant chance develop sca...,while having sex i felt a hard painful snap in...,you have had a penile fracture and there is a ...,"['sexual intercourse', 'penis', 'pain', 'lump']",1,[[ 3.36371452e-01 9.03653979e-01 -9.45858717e...,[[-0.6047527 -0.94729406 -0.91843134 -0.78592...,"[-0.6047527, -0.94729406, -0.91843134, -0.7859...","[3.36371452e-01, 9.03653979e-01, -9.45858717e-...","[-0.026996338999023824, -0.04228748639819485, ...","[0.01518952866458144, 0.0408063108069099, -0.0..."
1,spraying furniture rubbing alcohol cleaning ki...,necessary human head louse egg live long furni...,will spraying furniture with rubbing alcohol a...,this is not necessary human head lice and thei...,['lice'],1,[[ 6.9667971e-01 1.2845362e+00 -1.5401514e-01...,[[-0.1739369 -0.8185432 0.38514182 -0.95360...,"[-0.1739369, -0.8185432, 0.38514182, -0.953606...","[6.9667971e-01, 1.2845362e+00, -1.5401514e-01,...","[-0.007476340716820318, -0.035183493868387886,...","[0.03159803672819233, 0.058260376244189185, -0..."
2,neuroleptic drug parkinson disease,neuroleptic drug also called major tranquilize...,what are neuroleptic drugs for parkinson is di...,neuroleptic drugs also called major tranquiliz...,"[""parkinson's disease"", 'drug']",1,[[ 0.29717866 1.4708629 -0.56074375 -0.64780...,[[-2.93873221e-01 -8.65725994e-01 -3.97973090e...,"[-2.93873221e-01, -8.65725994e-01, -3.97973090...","[0.29717866, 1.4708629, -0.56074375, -0.647809...","[-0.012933558105495378, -0.03810118325424671, ...","[0.01300498955343724, 0.06436719463314898, -0...."
3,peanut help prevent headache told eat handful ...,apple day keep doctor away heard handful peanu...,do peanuts help prevent headaches have been to...,an apple a day keeps the doctor away but i hav...,"['peanut', 'headache']",1,[[ 0.22861195 0.99265194 -0.75949425 -0.22097...,[[-0.23108998 -0.8298196 -0.74065125 -0.43363...,"[-0.23108998, -0.8298196, -0.74065125, -0.4336...","[0.22861195, 0.99265194, -0.75949425, -0.22097...","[-0.010604766109877773, -0.03808058995631195, ...","[0.011121512637579711, 0.048290525037855705, -..."
4,osteoarthritis diagnosed,doctor make diagnosis oa based physical exam h...,how is osteoarthritis diagnosed,doctors make a diagnosis of oa based on a phys...,['osteoarthritis'],1,[[-0.10100315 0.81206137 -0.6272399 0.94998...,[[-0.6990545 -0.92379653 -0.74066365 0.95988...,"[-0.6990545, -0.92379653, -0.74066365, 0.95988...","[-0.10100315, 0.81206137, -0.6272399, 0.949985...","[-0.03207341160127254, -0.04238483028507408, -...","[-0.004589006824136842, 0.03689543512799267, -..."


In [ ]:
#normalizing validation question and answer embeddings
val_embeds['question_embeds_norm']=val_embeds['question_embeds'].apply(lambda x: normalize(x))
val_embeds['answer_embeds_norm']=val_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [ ]:
val_embeds.head()

,short_question,short_answer,short_question_pre,short_answer_pre,tags,label,short_answer_embed_numpy,short_question_embed_numpy,question_embeds,answer_embeds,question_embeds_norm,answer_embeds_norm
0,love keep cold better cloak love keep cold bet...,related mary poppins,love keeps the cold out better than a cloak lo...,are you related to mary poppins,"['cold', 'coldness']",1,[[-1.88111201e-01 6.26653075e-01 -9.74694312e...,[[-1.57408684e-01 -6.03189983e-04 8.07640851e...,"[-1.57408684e-01, -6.03189983e-04, 8.07640851e...","[-1.88111201e-01, 6.26653075e-01, -9.74694312e...","[-0.007677573286369261, -2.9420456244245898e-0...","[-0.008586915378059565, 0.02860551045243613, -..."
1,penis itch right month ago penis started itch ...,penis cancer,why does my penis itch all right about a mont...,penis cancer,"['finger', 'penis', 'urethra', 'chlamydia', 'd...",1,[[-2.58756757e-01 9.03034806e-02 -6.57845974e...,[[-0.5283349 -0.9306266 -0.7680437 -0.99272...,"[-0.5283349, -0.9306266, -0.7680437, -0.992728...","[-2.58756757e-01, 9.03034806e-02, -6.57845974e...","[-0.023301444961013686, -0.0410439372813632, -...","[-0.010805572487638368, 0.003771035071015924, ..."
2,type 2 diabetic eat unsweetened applesauce,diabetes well controlled unsweetened applesauc...,can a type 2 diabetic eat unsweetened applesauce,if your diabetes is well controlled unsweetene...,['diabetes'],1,[[-1.19244464e-01 7.17789233e-01 2.53438562e...,[[-6.07122898e-01 -9.22508121e-01 7.02447772e...,"[-6.07122898e-01, -9.22508121e-01, 7.02447772e...","[-1.19244464e-01, 7.17789233e-01, 2.53438562e-...","[-0.027863957824107718, -0.04233858986478362, ...","[-0.005694171538205783, 0.03427593100656782, 0..."
3,get pregnant boyfriend dick touched vagina hol...,correct le vulgar term male genitalia penis an...,can i get pregnant if my boyfriends dick touch...,the correct and less vulgar term for the male ...,"['pregnancy', 'vagina', 'penis']",1,[[ 1.01762474e+00 9.58866239e-01 4.19494271e...,[[ 0.11897181 -0.9848363 0.9506907 -0.92729...,"[0.11897181, -0.9848363, 0.9506907, -0.9272967...","[1.01762474e+00, 9.58866239e-01, 4.19494271e-0...","[0.005048763251720814, -0.041793138394723046, ...","[0.04568486443004084, 0.043046982265051985, 0...."
4,men daily skin,men need skin care regimen cleanse morning nig...,what should men do daily for their skin,men need skin care regimens too they should cl...,['skin'],1,[[ 0.32329905 0.895853 -0.87398374 -0.99327...,[[ 2.97095776e-02 -8.38858902e-01 -5.54627120e...,"[2.97095776e-02, -8.38858902e-01, -5.54627120e...","[0.32329905, 0.895853, -0.87398374, -0.9932766...","[0.0013287096526714756, -0.037516518589506824,...","[0.014299167148023612, 0.03962260881081586, -0..."


In [ ]:
#normalizing test question and answer embeddings
test_embeds['question_embeds_norm']=test_embeds['question_embeds'].apply(lambda x: normalize(x))
test_embeds['answer_embeds_norm']=test_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [ ]:
test_embeds.head()

,short_question,short_answer,short_question_pre,short_answer_pre,tags,label,short_answer_embed_numpy,short_question_embed_numpy,question_embeds,answer_embeds,question_embeds_norm,answer_embeds_norm
0,black spot right eye looking computer screen s...,one possibility would vitreous floater eye fil...,black spot in right eye when looking at comput...,one possibility would be a vitreous floater th...,['eyes'],1,[[ 0.47819456 1.0161952 -0.98465675 -0.97381...,[[-5.15649617e-01 -9.34034407e-01 -9.65542674e...,"[-5.15649617e-01, -9.34034407e-01, -9.65542674...","[0.47819456, 1.0161952, -0.98465675, -0.973817...","[-0.023089902518194434, -0.04182445345687039, ...","[0.021325880701919275, 0.04531891288153299, -0..."
1,severe pain swelling buttock groin thigh calf,depending age physical status activity would e...,i have severe pain and swelling in my buttock ...,depending on your age and physical status and ...,"['groin', 'severe pain and swelling in my butt...",1,[[ 0.32686195 0.95128304 -0.99937916 0.99864...,[[-8.20329726e-01 -9.76690352e-01 -9.99416888e...,"[-8.20329726e-01, -9.76690352e-01, -9.99416888...","[0.32686195, 0.95128304, -0.99937916, 0.998648...","[-0.03638637369248898, -0.0433218729047015, -0...","[0.014013913775523182, 0.04078540955494382, -0..."
2,vitamin present fish oil carrot carrot contain...,reason people take fish oil type fat provides ...,what vitamins are present in fish oil carrots ...,the reason people take fish oil is for the typ...,"['vitamin', 'fish', 'oil', 'carrot']",1,[[ 0.11415639 0.89580375 0.8809125 0.25639...,[[-0.5410029 -0.92638665 0.8828896 -0.53410...,"[-0.5410029, -0.92638665, 0.8828896, -0.534109...","[0.11415639, 0.89580375, 0.8809125, 0.25639057...","[-0.025460636952089142, -0.04359753741229866, ...","[0.005377251750770377, 0.04219616863352257, 0...."
3,experiencing headache make vomit i’m wondering...,although many reason cause symptom severe horm...,i have been experiencing headaches that make m...,although many reasons can cause such symptoms ...,['experiencing headaches that make me vomit'],1,[[ 6.92005157e-01 9.27820444e-01 -8.79468679e...,[[-0.574489 -0.9495875 -0.93554753 0.99138...,"[-0.574489, -0.9495875, -0.93554753, 0.9913813...","[6.92005157e-01, 9.27820444e-01, -8.79468679e-...","[-0.026745630882618023, -0.04420853448150973, ...","[0.03214307717911216, 0.04309650562307892, -0...."
4,happens swallow fly,1 le daily protein requirement depending size ...,what happens if you swallow a fly,you will have about 1 of less of your daily pr...,['swallow'],1,[[-0.0915726 0.77021825 0.13561723 0.93722...,[[-7.1134287e-01 -9.4363791e-01 4.7064021e-01...,"[-7.1134287e-01, -9.4363791e-01, 4.7064021e-01...","[-0.0915726, 0.77021825, 0.13561723, 0.937223,...","[-0.03218773606721418, -0.04269891394300137, 0...","[-0.00442513700271271, 0.0372198810369, 0.0065..."


In [ ]:
#final dataset for gpt training
train_gpt_data=train_embeds[['short_question_pre','short_answer_pre','question_embeds_norm','answer_embeds_norm']].copy()
train_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
train_gpt_data.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,while having sex i felt a hard painful snap in...,you have had a penile fracture and there is a ...,"[-0.026996338999023824, -0.04228748639819485, ...","[0.01518952866458144, 0.0408063108069099, -0.0..."
1,will spraying furniture with rubbing alcohol a...,this is not necessary human head lice and thei...,"[-0.007476340716820318, -0.035183493868387886,...","[0.03159803672819233, 0.058260376244189185, -0..."
2,what are neuroleptic drugs for parkinson is di...,neuroleptic drugs also called major tranquiliz...,"[-0.012933558105495378, -0.03810118325424671, ...","[0.01300498955343724, 0.06436719463314898, -0...."


In [ ]:
#saving gpt training data to disk
train_gpt_data.to_pickle(work_dir + 'data/gpt-ready/train_gpt_data.pkl')

In [ ]:
#final dataset for gpt validation
val_gpt_data=val_embeds[['short_question_pre','short_answer_pre','question_embeds_norm','answer_embeds_norm']].copy()
val_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
val_gpt_data.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,love keeps the cold out better than a cloak lo...,are you related to mary poppins,"[-0.007677573286369261, -2.9420456244245898e-0...","[-0.008586915378059565, 0.02860551045243613, -..."
1,why does my penis itch all right about a mont...,penis cancer,"[-0.023301444961013686, -0.0410439372813632, -...","[-0.010805572487638368, 0.003771035071015924, ..."
2,can a type 2 diabetic eat unsweetened applesauce,if your diabetes is well controlled unsweetene...,"[-0.027863957824107718, -0.04233858986478362, ...","[-0.005694171538205783, 0.03427593100656782, 0..."


In [ ]:
#saving gpt validation data to disk
val_gpt_data.to_pickle(work_dir + 'data/gpt-ready/val_gpt_data.pkl')

In [ ]:
#final dataset for gpt Test
test_gpt_data=test_embeds[['short_question_pre','short_answer_pre','question_embeds_norm','answer_embeds_norm']].copy()
test_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
test_gpt_data.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,black spot in right eye when looking at comput...,one possibility would be a vitreous floater th...,"[-0.023089902518194434, -0.04182445345687039, ...","[0.021325880701919275, 0.04531891288153299, -0..."
1,i have severe pain and swelling in my buttock ...,depending on your age and physical status and ...,"[-0.03638637369248898, -0.0433218729047015, -0...","[0.014013913775523182, 0.04078540955494382, -0..."
2,what vitamins are present in fish oil carrots ...,the reason people take fish oil is for the typ...,"[-0.025460636952089142, -0.04359753741229866, ...","[0.005377251750770377, 0.04219616863352257, 0...."


In [ ]:
#saving gpt test data to disk
test_gpt_data.to_pickle(work_dir + 'data/gpt-ready/test_gpt_data.pkl')

In [ ]:

train_gpt_data.shape,val_gpt_data.shape,test_gpt_data.shape

((21398, 4), (2377, 4), (2642, 4))

In [ ]:
#displaying the train gpt data
train_gpt_data.head(5)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,while having sex i felt a hard painful snap in...,you have had a penile fracture and there is a ...,"[-0.026996338999023824, -0.04228748639819485, ...","[0.01518952866458144, 0.0408063108069099, -0.0..."
1,will spraying furniture with rubbing alcohol a...,this is not necessary human head lice and thei...,"[-0.007476340716820318, -0.035183493868387886,...","[0.03159803672819233, 0.058260376244189185, -0..."
2,what are neuroleptic drugs for parkinson is di...,neuroleptic drugs also called major tranquiliz...,"[-0.012933558105495378, -0.03810118325424671, ...","[0.01300498955343724, 0.06436719463314898, -0...."
3,do peanuts help prevent headaches have been to...,an apple a day keeps the doctor away but i hav...,"[-0.010604766109877773, -0.03808058995631195, ...","[0.011121512637579711, 0.048290525037855705, -..."
4,how is osteoarthritis diagnosed,doctors make a diagnosis of oa based on a phys...,"[-0.03207341160127254, -0.04238483028507408, -...","[-0.004589006824136842, 0.03689543512799267, -..."


###FAISS COSINE SIMILARIY (To get similar QnAs)

In [ ]:
val_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/val_gpt_data.pkl')
test_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/test_gpt_data.pkl')

In [ ]:
train_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/train_gpt_data.pkl')
question_bert = train_gpt_QnA["Q_FFNN_embeds"].tolist()
answer_bert = train_gpt_QnA["A_FFNN_embeds"].tolist()
question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')
answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])
answer_index.add(answer_bert)

In [ ]:
answer_index.train

<faiss.swigfaiss.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x7f61013cb8a0> >

In [ ]:
#loading pre-trained gpt2 tokenizer and gpt2 model
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#defining function to prepare the actual gpt training data by retrieving similar question and answer pairs
#https://github.com/ash3n/DocProduct/blob/master/docproduct/train_embedding_to_gpt2_data.py
def train_embed_gpt2_data(question,answer,question_embedding):
  topk=20
  scores,indices=answer_index.search(
                  np.array([question_embedding]).astype('float32'), topk)
  q_sub=train_gpt_QnA.iloc[indices.reshape(20)]
  
  line = '`QUESTION: %s `ANSWER: %s' % (
                        question, answer)
  encoded_len=len(tokenizer.encode(line))
  for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')
    encoded_len=len(tokenizer.encode(line))
    if encoded_len>=1024:
      break
  return tokenizer.encode(line)[-1024:]

In [ ]:
#preparing the gpt data for the training data
tqdm.pandas()
train_gpt_QnA['gpt_data']=train_gpt_QnA.progress_apply(lambda x: train_embed_gpt2_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)

  0%|          | 0/21398 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1176 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
#preparing the gpt data for the validation data
val_gpt_QnA['gpt_data']=val_gpt_QnA.progress_apply(lambda x: train_embed_gpt2_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)
 

  0%|          | 0/2377 [00:00<?, ?it/s]

In [ ]:
#preparing the gpt data for the test data
test_gpt_QnA['gpt_data']=test_gpt_QnA.progress_apply(lambda x: train_embed_gpt2_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)

  0%|          | 0/2642 [00:00<?, ?it/s]

In [ ]:
#function to get the start of positive mask to be used while creating the masks
def mask_start(gpt_data):
  return 1024-gpt_data[::-1].index(4600)+1

In [ ]:
#finding the mask start for the train dataset
train_gpt_QnA['mask_start']=train_gpt_QnA.gpt_data.progress_apply(lambda x: mask_start(x))

  0%|          | 0/21398 [00:00<?, ?it/s]

In [ ]:
#finding the mask start for the validation dataset
val_gpt_QnA['mask_start']=val_gpt_QnA.gpt_data.progress_apply(lambda x: mask_start(x))
     

  0%|          | 0/2377 [00:00<?, ?it/s]

In [ ]:
#finding the mask start for the test dataset
test_gpt_QnA['mask_start']=test_gpt_QnA.gpt_data.progress_apply(lambda x: mask_start(x))

  0%|          | 0/2642 [00:00<?, ?it/s]

In [ ]:
#saving train data to disk
train_gpt_QnA.to_pickle(work_dir +'data/gpt-ready/train_gpt_QnA_inbuild.pkl')
     


In [ ]:
#saving val data to disk
val_gpt_QnA.to_pickle(work_dir +'data/gpt-ready/val_gpt_QnA_inbuild.pkl')

In [ ]:
#saving test data to disk
test_gpt_QnA.to_pickle(work_dir +'data/gpt-ready/test_gpt_QnA_inbuild.pkl')

In [ ]:
#train_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/train_gpt_QnA_inbuild.pkl')
# val_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/val_gpt_QnA_inbuild.pkl')
# test_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/test_gpt_QnA_inbuild.pkl')

In [ ]:

#finding length of the gpt data for train dataset
train_gpt_QnA['gpt_lens']=train_gpt_QnA.gpt_data.apply(lambda x: len(x))

In [ ]:
train_gpt_QnA.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds,gpt_data,mask_start,gpt_lens
0,while having sex i felt a hard painful snap in...,you have had a penile fracture and there is a ...,"[-0.026996338999023824, -0.04228748639819485, ...","[0.01518952866458144, 0.0408063108069099, -0.0...","[2147, 2073, 2499, 6078, 2911, 3387, 1037, 460...",927,1024
1,will spraying furniture with rubbing alcohol a...,this is not necessary human head lice and thei...,"[-0.007476340716820318, -0.035183493868387886,...","[0.03159803672819233, 0.058260376244189185, -0...","[290, 30523, 1117, 82, 1312, 1011, 262, 584, 1...",983,1024
2,what are neuroleptic drugs for parkinson is di...,neuroleptic drugs also called major tranquiliz...,"[-0.012933558105495378, -0.03810118325424671, ...","[0.01300498955343724, 0.06436719463314898, -0....","[922, 2882, 422, 616, 22356, 396, 1312, 561, 5...",968,1024


In [ ]:
#finding length of the gpt data for validation dataset
val_gpt_QnA['gpt_lens']=val_gpt_QnA.gpt_data.apply(lambda x: len(x))

In [ ]:
val_gpt_QnA.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds,gpt_data,mask_start,gpt_lens
0,love keeps the cold out better than a cloak lo...,are you related to mary poppins,"[-0.007677573286369261, -2.9420456244245898e-0...","[-0.008586915378059565, 0.02860551045243613, -...","[1309, 262, 40770, 2353, 351, 262, 827, 38229,...",1013,1024
1,why does my penis itch all right about a mont...,penis cancer,"[-0.023301444961013686, -0.0410439372813632, -...","[-0.010805572487638368, 0.003771035071015924, ...","[460, 407, 1283, 284, 3785, 503, 644, 340, 318...",1020,1024
2,can a type 2 diabetic eat unsweetened applesauce,if your diabetes is well controlled unsweetene...,"[-0.027863957824107718, -0.04233858986478362, ...","[-0.005694171538205783, 0.03427593100656782, 0...","[8726, 29, 2, 844, 3019, 18, 76, 5162, 37020, ...",985,1024


In [ ]:
#finding length of the gpt data for test dataset
test_gpt_QnA['gpt_lens']=test_gpt_QnA.gpt_data.apply(lambda x: len(x))

In [ ]:
test_gpt_QnA.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds,gpt_data,mask_start,gpt_lens
0,black spot in right eye when looking at comput...,one possibility would be a vitreous floater th...,"[-0.023089902518194434, -0.04182445345687039, ...","[0.021325880701919275, 0.04531891288153299, -0...","[3555, 15232, 290, 373, 1297, 1312, 550, 4151,...",945,1024
1,i have severe pain and swelling in my buttock ...,depending on your age and physical status and ...,"[-0.03638637369248898, -0.0433218729047015, -0...","[0.014013913775523182, 0.04078540955494382, -0...","[8620, 340, 220, 1312, 6686, 616, 3211, 16842,...",949,1024
2,what vitamins are present in fish oil carrots ...,the reason people take fish oil is for the typ...,"[-0.025460636952089142, -0.04359753741229866, ...","[0.005377251750770377, 0.04219616863352257, 0....","[428, 290, 991, 423, 262, 3663, 284, 1622, 205...",936,1024


In [ ]:
#finding number of train datapoints that has sequence length less than 1024
train_gpt_QnA[train_gpt_QnA.gpt_lens<1024].shape

(64, 7)

In [ ]:
#finding number of validation datapoints that has sequence length less than 1024
val_gpt_QnA[val_gpt_QnA.gpt_lens<1024].shape

(9, 7)

In [ ]:
#finding number of test datapoints that has sequence length less than 1024
test_gpt_QnA[test_gpt_QnA.gpt_lens<1024].shape

(5, 7)

In [ ]:
#finding train datapoints that has sequence length of 1024
train_gpt_data_cleaned=train_gpt_QnA[train_gpt_QnA.gpt_lens==1024]
train_gpt_data_cleaned.shape

(21334, 7)

In [ ]:
#finding validation datapoints that has sequence length of 1024
val_gpt_data_cleaned=val_gpt_QnA[val_gpt_QnA.gpt_lens==1024]
val_gpt_data_cleaned.shape

(2368, 7)

In [ ]:
#finding test datapoints that has sequence length of 1024
test_gpt_data_cleaned=test_gpt_QnA[test_gpt_QnA.gpt_lens==1024]
test_gpt_data_cleaned.shape

(2637, 7)

In [ ]:
#function to create the loss mask given mask start
def return_loss_mask(mask_start):
  return [0]*mask_start+[1]*(1024-mask_start)

In [ ]:
#creating loss mask for train data
train_gpt_data_cleaned['mask']=train_gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

In [ ]:
#creating loss mask for validation data
val_gpt_data_cleaned['mask']=val_gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

In [ ]:
#creating loss mask for test data
test_gpt_data_cleaned['mask']=test_gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

In [ ]:
#saving train gpt data with loss mask to disk
train_gpt_data_cleaned.to_pickle(work_dir +'data/gpt-ready/train_gpt_inbuild_cleaned.pkl')
#saving validation gpt data with loss mask to disk
val_gpt_data_cleaned.to_pickle(work_dir +'data/gpt-ready/val_gpt_inbuild_cleaned.pkl')
#saving test gpt data with loss mask to disk
test_gpt_data_cleaned.to_pickle(work_dir +'data/gpt-ready/test_gpt_inbuild_cleaned.pkl')

NameError: ignored

In [ ]:
#Uncomment if loading from disk
#loading gpt train, validation and test data from disk
train_gpt_data_cleaned=pd.read_pickle(work_dir +'data/gpt-ready/train_gpt_inbuild_cleaned.pkl')
val_gpt_data_cleaned=pd.read_pickle(work_dir +'data/gpt-ready/val_gpt_inbuild_cleaned.pkl')
test_gpt_data_cleaned=pd.read_pickle(work_dir +'data/gpt-ready/test_gpt_inbuild_cleaned.pkl')


In [ ]:
train_gpt_data_cleaned.shape,val_gpt_data_cleaned.shape,test_gpt_data_cleaned.shape

((21334, 8), (2368, 8), (2637, 8))

In [ ]:
# transform train gpt data to numpy array
train_gpt_data=[]
for i in train_gpt_data_cleaned.gpt_data.values:
  train_gpt_data.append(i)
train_gpt_data=np.array(train_gpt_data)
train_gpt_data.shape

(21334, 1024)

In [ ]:
# transform train loss mask to numpy array
train_loss_mask=[]
for i in train_gpt_data_cleaned['mask'].values:
  train_loss_mask.append(i)
train_loss_mask=np.array(train_loss_mask)
train_loss_mask.shape

(21334, 1024)

In [ ]:
# transform validation gpt data to numpy array
val_gpt_data=[]
for i in val_gpt_data_cleaned.gpt_data.values:
  val_gpt_data.append(i)
val_gpt_data=np.array(val_gpt_data)
val_gpt_data.shape

(2368, 1024)

In [ ]:
# transform val loss mask to numpy array
val_loss_mask=[]
for i in val_gpt_data_cleaned['mask'].values:
  val_loss_mask.append(i)
val_loss_mask=np.array(val_loss_mask)
val_loss_mask.shape

(2368, 1024)

In [ ]:
# transform test gpt data to numpy array
test_gpt_data=[]
for i in test_gpt_data_cleaned.gpt_data.values:
  test_gpt_data.append(i)
test_gpt_data=np.array(test_gpt_data)
test_gpt_data.shape

(2637, 1024)

In [ ]:
# transform test loss mask to numpy array
test_loss_mask=[]
for i in test_gpt_data_cleaned['mask'].values:
  test_loss_mask.append(i)
test_loss_mask=np.array(test_loss_mask)
test_loss_mask.shape

(2637, 1024)

In [ ]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt2 train dataset

BATCH_SIZE = 1
BUFFER_SIZE = 20000
train_gpt_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': train_gpt_data,

        'label': train_gpt_data[:,1:],
        'loss_mask':train_loss_mask[:,:-1]
    },
))


train_gpt_dataset = train_gpt_dataset.shuffle(BUFFER_SIZE)
train_gpt_dataset = train_gpt_dataset.batch(BATCH_SIZE)
train_gpt_dataset = train_gpt_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#displaying sample gpt training data
next(iter(train_gpt_dataset))

({'context': <tf.Tensor: shape=(1, 1024), dtype=int64, numpy=array([[17372,   680,   379, ...,  3393,   922,  8458]])>,
  'label': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[ 680,  379, 1661, ..., 3393,  922, 8458]])>,
  'loss_mask': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[0, 0, 0, ..., 1, 1, 1]])>},)

In [ ]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt2 validation dataset

BATCH_SIZE = 1
BUFFER_SIZE = 20000
val_gpt_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': val_gpt_data,

        'label': val_gpt_data[:,1:],
        'loss_mask':val_loss_mask[:,:-1]
    },
))

val_gpt_dataset = val_gpt_dataset.shuffle(BUFFER_SIZE)
val_gpt_dataset = val_gpt_dataset.batch(BATCH_SIZE)
val_gpt_dataset = val_gpt_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#displaying sample gpt validation data
next(iter(val_gpt_dataset))

({'context': <tf.Tensor: shape=(1, 1024), dtype=int64, numpy=array([[ 286,  790, 2278, ...,  612,  922, 8458]])>,
  'label': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[ 790, 2278, 4600, ...,  612,  922, 8458]])>,
  'loss_mask': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[0, 0, 0, ..., 1, 1, 1]])>},)

In [ ]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt2 test dataset

BATCH_SIZE = 1
BUFFER_SIZE = 20000
test_gpt_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': test_gpt_data,

        'label': test_gpt_data[:,1:],
        'loss_mask':test_loss_mask[:,:-1]
    },
))

test_gpt_dataset = test_gpt_dataset.shuffle(BUFFER_SIZE)
test_gpt_dataset = test_gpt_dataset.batch(BATCH_SIZE)
test_gpt_dataset = test_gpt_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#displaying sample gpt test data
next(iter(test_gpt_dataset))

({'context': <tf.Tensor: shape=(1, 1024), dtype=int64, numpy=array([[1933,  706,  281, ..., 1029, 2910, 3833]])>,
  'label': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[ 706,  281,  300, ..., 1029, 2910, 3833]])>,
  'loss_mask': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[0, 0, 0, ..., 1, 1, 1]])>},)

In [ ]:
#training the gpt2 model
learning_rate=0.0001
optim=tf.optimizers.Adam(learning_rate)
j=0
loss_cum=0
epoch_loss=0
stop_training=False
epochs=10
for epoch in range(epochs):
  print('epoch',epoch)
  for i in tqdm(train_gpt_dataset):
    with tf.GradientTape() as tape:
      context=i[0]['context']
      loss_mask=tf.cast(i[0]['loss_mask'],tf.float32)
      output=tf_gpt2_model(context,training=True)['logits']
      loss_value = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=np.array(i[0]['label']), logits=output[ :,:-1])*loss_mask)
      j=j+1
      loss_cum+=loss_value
      epoch_loss+=loss_value/100
      if j%50==0:
        print(loss_cum/50)
        grads=tape.gradient(loss_cum/50,tf_gpt2_model.trainable_variables)
        optim.apply_gradients(zip(grads,tf_gpt2_model.trainable_variables))
        loss_cum=0
      if tf.math.is_nan(loss_value):
        print('loss became nan')
        stop_training=True
        break
      if j%10000==0:
        tf_gpt2_model.save_pretrained(work_dir + "gpt2_model_new/medical_gpt2_model_new_2_10"+str(epoch)+'_'+str(j))
  print('epoch_loss by 100',epoch_loss/237)
  epoch_loss=0
  if stop_training:
    break
  tf_gpt2_model.save_pretrained(work_dir + "gpt2_model_new/medical_gpt2_model_new_2_10"+str(epoch))

epoch 0


  0%|          | 0/21334 [00:00<?, ?it/s]

tf.Tensor(481.40866, shape=(), dtype=float32)


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

tf.Tensor(386.02716, shape=(), dtype=float32)


...

tf.Tensor(341.46844, shape=(), dtype=float32)


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=4265>

tf.Tensor(348.3703, shape=(), dtype=float32)


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=4266>

epoch_loss by 100 tf.Tensor(312.74655, shape=(), dtype=float32)


In [ ]:
#loading the model from the saved checkpoint above
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained(work_dir + "./medical_gpt2_model_new_2_102_50000")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/./medical_gpt2_model_new_2_102_50000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#function to return sample origingal and generated answers for the train dataset
def return_sample_output_predicted(gpt_dataset):
  sample_data=next(iter(gpt_dataset))
  v=sample_data[0]['context'].numpy()[0]
  mask_start = len(v) - list(v[::-1]).index(4600) + 1
  input=v[:mask_start+1]
  next_step_input=[]
  input1=list(input)+next_step_input
  final_output=[]
  for i in range(0,1024-len(input)):
    input1=list(input1)+next_step_input
    output=tf_gpt2_model([np.array(input1)])['logits']
    output1=np.argmax(output,axis=1)
    next_step_input=[output1[-1]]#appending last output to input
    next_step_char=tokenizer.decode(next_step_input)
    final_output.append(next_step_char)
  expected_output=v[mask_start+1:]
  return v,tokenizer.decode(expected_output),' '.join(final_output)

In [ ]:
#displaying sample original context, expected answer and generated answer for train datset
for i in range(5):
  v,exp_ans,fin_ans=return_sample_output_predicted(train_gpt_dataset)
  print(tokenizer.decode(v),end='\n')
  print(exp_ans,end='\n')
  print(fin_ans,end='\n')
  print('------------------')

 in previous years e coli can cause severe stomach cramps and diarrhea nausea and vomiting `QUESTION: what are the symptoms of acid reflux disease `ANSWER: could acid reflux also cause upper back pain `QUESTION: what are the symptoms of gastroenteritis `ANSWER: does crackers help with nausea `QUESTION: what are the symptoms of gallstones `ANSWER: can having gallbladder symptoms include losing weight over approximately a one year period especially if i been dieting the right way watching label is and portions stopped soft drinks no more frozen dinners or fast food i have known i have a gallstone for about 7 years but no symptoms in fact unless losing weight is a symptom this was found while checking something else the size had more than doubled since the last check 10 something thank you `QUESTION: how do you get rid of a chest cold `ANSWER: drinking of lukewarm water hot fomentation and consuming herbal tea prepared with herbs like yastimadhu tulsi haridra green tea and dalchini `QUEST

In [ ]:
# #function to return sample origingal and generated answers for the validation dataset
# def return_validation_sample_output_predicted():
#   sample_data=next(iter(val_gpt_dataset))
#   v=sample_data[0]['context'].numpy()[0]
#   mask_start = len(v) - list(v[::-1]).index(4600) + 1
#   input=v[:mask_start+1]
#   next_step_input=[]
#   input1=list(input)+next_step_input
#   final_output=[]
#   token_output=[]
#   for i in range(0,1024-len(input)):
#     input1=list(input1)+next_step_input
#     output=tf_gpt2_model([np.array(input1)])['logits']
#     output1=np.argmax(output,axis=1)
#     next_step_input=[output1[-1]]#appending last output to input
#     next_step_char=tokenizer.decode(next_step_input)
#     token_output.append(next_step_input[0])
#     final_output.append(next_step_char)
#   expected_output=v[mask_start+1:]
#   return v,tokenizer.decode(expected_output),tokenizer.decode(token_output)

In [ ]:

#displaying sample original context, expected answer and generated answer for validation datset
for i in range(5):
  v,exp,fin=return_sample_output_predicted(val_gpt_dataset)
  print(tokenizer.decode(v))
  print(exp)
  print(fin)
  print('------------------')

 for a nice and easy power walk for about 50 minutes i come home and suddenly i just feel sluggish and exhausted i do not starve myself i eat when i am hungry and when i need to eat i get super tired and when i come home i just knock out because i get so tired i do not like to exercise when i have school it is been affecting my health and my body i really want to exercise daily help please <negative_smiley> `ANSWER: i like yoga i always feel rejuvenated after yoga `QUESTION: can burning a nerve help relieve nerve pain  a friend of mine was told burning the nerve can help relieve nerve pain `ANSWER: i have neuropathy and my doctor wanted to do this but stated that it is a 5050 chance that it would work my wife is a nurse and she asked about this with doctors she assists at the hospital and they stated the same and also the pain usually returns `QUESTION: treatment for ruptured sciatica chronic pain methadone percoset implant no help daughter is visit to local er for severe migraine was 

In [ ]:
#displaying sample original context, expected answer and generated answer for test datset
for i in range(5):
  v,exp,fin=return_sample_output_predicted(test_gpt_dataset)
  print(tokenizer.decode(v))
  print(exp)
  print(fin)
  print('------------------')

 like if you had sex or what other symptoms have you experienced give the details `QUESTION: what are the symptoms of generalized anxiety disorder gad `ANSWER: tension pressure confusion stress arguing `QUESTION: i have pain in my knee under the knee cap the back of my knee and my thigh is this just arthritis  some times i can hardly stand i have always tucked my leg under me when i sit down i can not any more the pain is too harsh some times my thigh muscles and calf muscles hurt so bad is this all part of arthris `ANSWER: i have had no previous injuries `QUESTION: im having lower back pains and im having pains in my pelvic area and i am leaking urine for no reason  i have to use the restroom about every hour i feel sick every time i wake up i have no idea whats going on i have never had a kid before i dont know what kind of doctor i need to see `ANSWER: lower back pain and front left pelvic area pain in the mornings hard to stand and walk urinating uncontrolled neck pain leftside aft

###Evaluation Metrics  
Bleu Score

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

###Final Prediction

In [ ]:
import re
#preprocessing questions and answers.
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

In [ ]:

def preprocess(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'
    
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text


In [ ]:
# l
# biobert_tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/BioRedditBERT-uncased")
# question_extractor_model=tf.keras.models.load_model(work_dir + 'question_extractor_model')

In [ ]:
from transformers import AutoTokenizer, TFAutoModel
def return_biobert_tokenizer_model():
    '''returns pretrained biobert tokenizer and question extractor model'''
    biobert_tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/BioRedditBERT-uncased")
    question_extractor_model=tf.keras.models.load_model(work_dir + 'question_extractor_model')
    return biobert_tokenizer,question_extractor_model

In [ ]:
def return_gpt2_tokenizer_model():
    '''returns pretrained gpt2 tokenizer and gpt2 model'''
    gpt2_tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
    tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained(work_dir + "gpt2_model/medical_gpt2_model_new_2_104_90000")
    return gpt2_tokenizer,tf_gpt2_model

In [ ]:
train_gpt_QnA = pd.read_pickle(work_dir + 'data/gpt-ready/train_gpt_data.pkl')
question_bert = train_gpt_QnA["Q_FFNN_embeds"].tolist()
answer_bert = train_gpt_QnA["A_FFNN_embeds"].tolist()
question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')

answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])

question_index = faiss.IndexFlatIP(question_bert.shape[-1])
answer_index.add(answer_bert)
question_index.add(question_bert)

In [ ]:
#defining function to prepare the data for gpt inference
#https://github.com/ash3n/DocProduct
def preparing_gpt_inference_data(question,question_embedding,gpt2_tokenizer):
  topk=20
  scores,indices=answer_index.search(
                  question_embedding.astype('float32'), topk)
  q_sub=train_gpt_QnA.iloc[indices.reshape(20)]
  
  line = '`QUESTION: %s `ANSWER: ' % (
                        question)
  encoded_len=len(gpt2_tokenizer.encode(line))
  for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')
    encoded_len=len(gpt2_tokenizer.encode(line))
    if encoded_len>=1024:
      break
  return gpt2_tokenizer.encode(line)[-1024:]

In [ ]:
#function to generate answer given a question and the required answer length
import random
def predict_answer(question,answer_len=25):
  preprocessed_question=preprocess(question)
  question_len=len(preprocessed_question.split(' '))
  truncated_question=preprocessed_question
  biobert_tokenizer,question_extractor_model= return_biobert_tokenizer_model()
  gpt2_tokenizer,tf_gpt2_model= return_gpt2_tokenizer_model()
  if question_len>500:
    truncated_question=' '.join(preprocessed_question.split(' ')[:500])
  encoded_question= biobert_tokenizer.encode(truncated_question)
  max_length=512
  padded_question=tf.keras.preprocessing.sequence.pad_sequences(
      [encoded_question], maxlen=max_length, padding='post')
  question_mask=[[1 if token!=0 else 0 for token in question] for question in padded_question]
  embeddings=question_extractor_model({'question':np.array(padded_question),'question_mask':np.array(question_mask)})
  gpt_input=preparing_gpt_inference_data(truncated_question,embeddings.numpy(),gpt2_tokenizer)
  mask_start = len(gpt_input) - list(gpt_input[::-1]).index(4600) + 1
  input=gpt_input[:mask_start+1]
  if len(input)>(1024-answer_len):
   input=input[-(1024-answer_len):]
  output_sequences = tf_gpt2_model.generate(
            input_ids =tf.constant([np.array(input)]),
            max_length = 1024,
            # temperature = 0.7,
            num_beams=3, 
            no_repeat_ngram_size=2, 
            #num_return_sequences=1, 
            early_stopping=True,
            # top_k = 0,
            # top_p = 0.9,
            # repetition_penalty = 1,
            # do_sample = True,
            num_return_sequences = 3,
            # do_sample=True, 
            # top_k=0,
            pad_token_id=gpt2_tokenizer.eos_token_id
        )
  #gpt2_output=gpt2_tokenizer.decode(tf_gpt2_model.generate(input_ids=tf.constant([np.array(input)]),max_length=1024,temperature=0.7,pad_token_id=gpt2_tokenizer.eos_token_id)[0])
  gpt2_output=gpt2_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
  #print(gpt2_output.find('`ANSWER : '))
  if gpt2_output.find('`ANSWER : ') != -1:
    answer=gpt2_output.rindex('`ANSWER : ')
    ansIndex = answer+len('`ANSWER : ')
  elif gpt2_output.find('`ANSWER') != -1:
    answer=gpt2_output.rindex('`ANSWER')
    ansIndex = answer+len('`ANSWER')
  else:
    answer=gpt2_output.rindex('`ANSWER: ')
    ansIndex = answer+len('`ANSWER: ')

  return gpt2_output[ansIndex:]
  #return gpt2_output,
  

In [ ]:
#function to generate answer given a question and the required answer length
import random
def predict_answer(question,answer_len=50):
  preprocessed_question=preprocess(question)
  question_len=len(preprocessed_question.split(' '))
  truncated_question=preprocessed_question
  biobert_tokenizer,question_extractor_model= return_biobert_tokenizer_model()
  gpt2_tokenizer,tf_gpt2_model= return_gpt2_tokenizer_model()
  if question_len>500:
    truncated_question=' '.join(preprocessed_question.split(' ')[:500])
  encoded_question= biobert_tokenizer.encode(truncated_question)
  max_length=512
  padded_question=tf.keras.preprocessing.sequence.pad_sequences(
      [encoded_question], maxlen=max_length, padding='post')
  question_mask=[[1 if token!=0 else 0 for token in question] for question in padded_question]
  embeddings=question_extractor_model({'question':np.array(padded_question),'question_mask':np.array(question_mask)})
  gpt_input=preparing_gpt_inference_data(truncated_question,embeddings.numpy(),gpt2_tokenizer)
  mask_start = len(gpt_input) - list(gpt_input[::-1]).index(4600) + 1
  input=gpt_input[:mask_start+1]
  if len(input)>(1024-answer_len):
   input=input[-(1024-answer_len):]
  output_sequences = tf_gpt2_model.generate(
            input_ids =tf.constant([np.array(input)]),
            max_length = 1024,
            top_k = 20, 
            top_p = 0.85,
            #temperature = 0.7,
            # num_beams=3, 
            #no_repeat_ngram_size=2, 
            # num_return_sequences=1, 
            early_stopping=True,
            # top_k = 0,
            # top_p = 0.9,
            # repetition_penalty = 1,
            do_sample = True,
            num_return_sequences = 3,
            # do_sample=True, 
            # top_k=0,
            pad_token_id=gpt2_tokenizer.eos_token_id
        )
  #gpt2_output=gpt2_tokenizer.decode(tf_gpt2_model.generate(input_ids=tf.constant([np.array(input)]),max_length=1024,temperature=0.7,pad_token_id=gpt2_tokenizer.eos_token_id)[0])
  #gpt2_output=gpt2_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
  #print(gpt2_output.find('`ANSWER : '))
  print("Output:\n" + 100 * '-')
  for i, sample_output in enumerate(output_sequences):
    gpt2_output = gpt2_tokenizer.decode(sample_output, skip_special_tokens = True)
    answer=gpt2_output.rindex('`ANSWER: ')
    print("{}: {}...".format(i, gpt2_output[answer:]))
    print('')
  #return gpt2_output,
  

In [ ]:
predict_answer('what are the signs of heart attack?')
#print("answer:",answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1173 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: it is common for a heart attack to be seen on the outside of a chest or outside of the chest if you see it on the outside it is very unlikely that it is something very serious if it is just your chest it would be best to...

1: `ANSWER: it may be possible to get heart attack from an infected heart node   a stroke that is usually not very serious and could lead to bleeding heart attack but if the heart fails the blood is still flowing and there is no pain or bleeding heart attack...

2: `ANSWER: a large heart attack usually begins when the heart stops heart rate drops and heart rate is increased rapidly heart rate is an oscillator in your blood vessels and your heart rate goes very slowly but not really heart rate is very good it keeps it going until...



In [ ]:
def calc_bleu_score(question,answer):
  answer_len=25
  generated_answer=predict_answer(question,answer_len)
  reference = [answer.split(' ')]
  candidate = generated_answer.split(' ')
  #print(generated_answer)
  score = sentence_bleu(reference, candidate)
  return score

In [ ]:
metric=calc_bleu_score('How do i know if i am pregnant','go to your doctor and ask for a test to confirm')
print(metric)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1112 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: hi if you are not sure if you are pregnant then you can go to your doctor and ask for a test to confirm...



AttributeError: ignored

In [ ]:
answer=predict_answer('what are the signs of heart attack?')
print("answer:",answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1173 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: heart attack occurs in one of the few groups of individuals in the world with a heart attack there are no signs of heart attack symptoms usually with a stroke it is usually just a vague sensation of something but then it is very dangerous and it can spread...

1: `ANSWER: heart attacks are very rare these symptoms are usually caused by a virus but not all are the same but the most common symptoms are irregular heartbeat and irregular heartbeat and the body is very sensitive to these abnormal events there are many different causes of these attacks so...

2: `ANSWER: there are no signs of heart attack in men or in women this is not a sign of heart attack in fact it is a sign of a heart attack but it is also a sign of heart failure as the blood pressure rises so the blood pressure is...

answer: None


In [ ]:
answer=predict_answer('How do i know if i am pregnant?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1112 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: your doctor may be able to help you figure out what your doctor may have prescribed you or your baby and give you a good shot at the end of this time and if you want to continue with it you can call your doctor or your pharmacist...

1: `ANSWER: you have been in the news right now it seems that you may be pregnant if you have been having unprotected sex before or during the pregnancy then it is important to note this you may be in the right place for you to be able to see your...

2: `ANSWER: the best way to tell is to go to your doctor to be tested for a pregnancy test your doctor is not the one to test for a pregnancy test just ask your doctor if you know you are pregnant if you know you are pregnant you will be...

None


In [ ]:
answer=predict_answer('How to treat osteoarthritis?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1386 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: hi there are different ways to treat the disease of the bone in your body but to be able to do that would be a great idea hope this helps you good luck ps if your feeling it i am sure that it is a side effect of f...

1: `ANSWER: you should do your primary medical history and your primary education level before getting your primary care provider to check the status of the fracture in your right hip i think that your primary care provider should know your specific medical history and your history will give you a...

2: `ANSWER: there are two main methods of treating osteoarthritis: surgery and treatment often with oral antibiotics or oral medications can also be used for the disease in addition to treatment oral antibiotics can be helpful for the symptoms of the disease     ...

None


In [ ]:
answer=predict_answer('How to stop migraine attack?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1119 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: there are no specific techniques to stop migraine attacks in addition to an individualized treatment plan i suggest you to take a medication called nsaids it is known to be effective for several months in treating migraine so i suggest you to try to get a...

1: `ANSWER: if you have an active or unconscious brain or nervous system in a close proximity to an object such as a car you may be more likely to stop it with a simple motion that will make your brain bleed but the only thing to do is to go...

2: `ANSWER: you will need to take your medicines to stop the migraines you may also need to take the ibuprofen fluoxetine if you have been taking the fluoxetine it may help to calm down the situation   ...

None


In [ ]:
answer=predict_answer('what should i do if my baby is vomiting?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1165 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: the baby is usually vomiting when his or her baby is breathing or when it is still in the air and if he or she is breathing normally the doctor should take him or her to the doctor for a proper exam to rule out any possible cause and...

1: `ANSWER: no baby is vomiting is not contagious it is important to see your doctor if it is an emergency call to a cbc ct cell provider or other emergency care provider may be able to help you to get a cbc ct ct c...

2: `ANSWER: vomiting is not always contagious so if your baby is vomiting he has been vomiting in the past few days so you need to see your doctor he can see what it is like for you if it is a viral or bacterial infection you need to take him...

None


In [ ]:
answer=predict_answer('why my eyes hurts sometimes?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1089 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: hi eye damage can affect your eyes or the eyes are closed but there is no way to know what you are looking at but i know that eye damage can be a little bit painful and even if it was just a minor eye problem its just a...

1: `ANSWER: the most common reason is because the eyes are not fully developed and not fully protected there are no eye protection to prevent damage but it would be best to get an eye exam done to rule out any possible medical problem    if your eyes are...

2: `ANSWER: i have had eye contact lenses for about 2 years and my hair is still soft and thin and there is nothing to worry about but i am sorry your eyes are still on the same color so i recommend going to the doctor to get it sorted out...

None


In [ ]:
answer=predict_answer('what should i do to avoid backaches')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1049 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: hi you have been diagnosed with backaches i know you have a couple of different types of backaches but you do need to get a proper diagnosis done if the symptoms of your backache persist and are still there hope this helps you good luck ps...

1: `ANSWER: most helpful answer to your question the answer is that most people with backaches should never have backaches so do a backache or you may be going to get back to your doctor and ask your doctor for a backache and he can give you...

2: `ANSWER: you may have an infection or even a blood test if your medical provider knows the type and severity of the infection and how long you have to be treated for it or not the doctor can help you make an appointment with your doctor as soon as possible...

None


In [ ]:
answer=predict_answer('How to prevent stomach flu?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1163 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: when you get the flu and you do not know any other symptoms and your symptoms you should be on your own do not be afraid or scared about it it and do not be afraid if it gets into your stomach and gets into the stomach do not...

1: `ANSWER: preventing stomach flu from developing is a very important goal so you need to do your best to do that to prevent stomach flu from spreading to other people in your household and also to keep yourself safe by doing this as much as possible avoid eating foods that...

2: `ANSWER: if you are having stomach flu then your first step is to go see your doctor as soon as possible if possible if you should be fine if not then you can still get your stomach flu to a maximum and avoid it again by going to a food...

None


In [ ]:
answer=predict_answer('Is conjunctivitis contagious?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1079 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: conjunctivitis contagious can spread to others if you are in a school or college dormitory and have been treated by a medical professional it is usually the school or college nurse who is most likely to spread the virus so it would not be wise...

1: `ANSWER: i am not aware of any contagious diseases or any other illnesses that could be spread through direct contact with a direct contact lens or other lens to see if it spreads further i am sure you will be fine and will take the appropriate precautions to avoid the...

2: `ANSWER: yes conjunctivitis is contagious in the u s and other countries so if you do not have symptoms you should see your medical provider immediately for an examination to confirm your diagnosis you may also need to talk to your pharmacist and get an anti...

None


In [ ]:
answer=predict_answer('how to diagnose cancer?')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1055 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: the most common test is the diphtheria c virus if the disease is present you may be diagnosed by your primary care provider or your primary care provider may recommend that you go to a gi clinic    where you can be examined...

1: `ANSWER: i have seen a lot of doctors over the internet and know that i do not have a problem with my doctor is opinion regarding the size of the lump on my chest i have had it for almost two years now with a lump in my chest but...

2: `ANSWER: there are a variety of conditions including pancreatic cancer in addition there are several other conditions that can cause cancer and they are very rare in the u s and in many countries this disease can be very serious so a medical provider can be able to help...

None


In [ ]:
answer=predict_answer('what are the alzheimer disease treatment')
print(answer)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/gdrive/My Drive/MSDS/498-Capstone/NW-MSDS_CapstoneProject/gpt2_model/medical_gpt2_model_new_2_104_90000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1214 > 1024). Running this sequence through the model will result in indexing errors


Output:
----------------------------------------------------------------------------------------------------
0: `ANSWER: alzheimer is disease is an extremely rare disease diagnosed with a combination of drugs that are both effective and preventative of disease and are associated with a lower risk population of being affected by other diseases the treatment usually involves using steroids and a prescription medication...

1: `ANSWER: there are many different treatments available to treat your symptoms and symptoms of alzheimer disease treatment is not always the most effective treatment because many patients with alzheimer disease often have symptoms similar to the symptoms of the disease and often do not respond to...

2: `ANSWER: it depends on the severity of the disease and whether the treatment is given to your particular type of disease or to other people in the same condition i recommend a biopsy for a blood test and for the blood test if possible if you are unsure about...

None
